# Import Library

In [1]:
import pandas as pd
import os
import MySQLdb
import csv
import sys
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
category_df = pd.read_csv("categories.csv")

category_df = category_df.fillna(0)
category_df["parent_cat_id"] = category_df["parent_cat_id"].astype(int)
category_df.head()

,cat_id,cat_title,parent_cat_id
0,8260,Ana_kategoriler,0
1,10321,Bilim,8260
2,25205,Teknoloji,10321
3,19758,Mühendislik,25205
4,25205,Teknoloji,19758


# Create Graph

In [3]:
G = nx.DiGraph()
G.add_node(0)

for i in range(category_df.shape[0]):

    cat_id = category_df.loc[i, "cat_id"]
    parent_cat_id = category_df.loc[i, "parent_cat_id"]
    
    G.add_node(cat_id)
    G.add_edge(cat_id, parent_cat_id)

In [4]:
children = [node for node in nx.dfs_postorder_nodes(G, 1090)]
print(children)

[0, 19439, 27349, 65533, 5165661, 1222276, 788638, 40683, 13088, 1023724, 16842, 1026359, 13244, 508015, 10325, 23078, 48135, 11087, 23103, 17449, 11939, 39595, 1723872, 1723868, 1723865, 5977611, 8620, 48139, 779099, 12059, 57361, 47843, 13488, 25247, 13504, 26952, 8972, 39849, 4363366, 26928, 4933361, 22814, 12719, 10327, 22816, 5912320, 10338, 5910034, 13494, 16687, 21766, 11884, 53357, 14779, 30640, 12661, 5975134, 19758, 19759, 12664, 22815, 86533, 18614, 14099, 20415, 20021, 4719776, 4524227, 18607, 18611, 61954, 18952, 47828, 18872, 11885, 55996, 16589, 16600, 26931, 13503, 25290, 40742, 26929, 12437, 19253, 27689, 18760, 4230421, 1899421, 15925, 16585, 5159096, 87091, 22869, 22867, 4465863, 4618165, 17454, 21369, 87056, 87105, 13109, 39864, 7774, 1899439, 16644, 18936, 51115, 51314, 48325, 68939, 17522, 2788607, 46543, 2788608, 13239, 3869307, 17240, 12539, 12529, 755231, 25223, 12670, 12524, 12543, 12551, 9582, 25222, 53311, 53312, 48735, 12665, 74614, 27745, 29255, 64757, 228

# Get Child Nodes

In [5]:
node = 1742828
list(G.predecessors(node))

[]

# Get Parent Nodes

In [6]:
list(G.successors(node))

[8573, 4645829, 28182, 15910, 64359]

In [7]:
no_child = []
yes_child = []

for i in range(category_df.shape[0]):
    cat_id = category_df.loc[i, "cat_id"]
    if(len(list(G.predecessors(cat_id))) == 0):
        no_child.append(cat_id)
    else:
        yes_child.append(cat_id)

In [8]:
len(yes_child) + len(no_child) == category_df.shape[0]

True

# Connect DB

In [9]:
conn = MySQLdb.connect(
    user='root',
    passwd='passwd',
    host='localhost',
    database='wiki',
    charset='utf8',
    init_command='SET NAMES UTF8'
)

c = conn.cursor() 


sql = 'select  cl_from,  ' \
            'cast(categorylinks.cl_to as char) as cl_to, ' \
            'cast(categorylinks.cl_sortkey as char) as childs ' \
            'from categorylinks ' \
            'where categorylinks.cl_type = "page" and categorylinks.cl_to = %s '

cl_to = "Alman_arkeologlar"

c.execute(sql, args = [cl_to])
for row in c.fetchall():
    print({'cl_from': row[0], 'cl_to': row[1], 'childs': row[2]})

{'cl_from': 14239, 'cl_to': 'Alman_arkeologlar', 'childs': 'MANFRED KORFMANN'}
{'cl_from': 36112, 'cl_to': 'Alman_arkeologlar', 'childs': 'WIEGAND, THEODOR\nTHEODOR WIEGAND'}
{'cl_from': 36202, 'cl_to': 'Alman_arkeologlar', 'childs': 'SCHLIEMANN, HEINRICH\nHEINRICH SCHLIEMANN'}
{'cl_from': 186047, 'cl_to': 'Alman_arkeologlar', 'childs': 'BLUM, STEPHAN\nSTEPHAN BLUM'}
{'cl_from': 552950, 'cl_to': 'Alman_arkeologlar', 'childs': 'ERNST EMIL HERZFELD'}
{'cl_from': 1133838, 'cl_to': 'Alman_arkeologlar', 'childs': 'WOLFGANG MÜLLER-WIENER'}
{'cl_from': 1577892, 'cl_to': 'Alman_arkeologlar', 'childs': 'EBERHARD ZANGGER'}
{'cl_from': 1751796, 'cl_to': 'Alman_arkeologlar', 'childs': 'KLAUS SCHMIDT (ARKEOLOG)'}
{'cl_from': 1889251, 'cl_to': 'Alman_arkeologlar', 'childs': 'MAX VON OPPENHEIM'}
{'cl_from': 2082912, 'cl_to': 'Alman_arkeologlar', 'childs': 'KRAUTHEIMER, RICHARD\nRICHARD KRAUTHEIMER'}
{'cl_from': 2126995, 'cl_to': 'Alman_arkeologlar', 'childs': 'ALFONS MARIA SCHNEIDER'}
{'cl_from': 232

In [10]:
sql = 'select  cl_from,  ' \
            'cast(categorylinks.cl_to as char) as cl_to, ' \
            'cast(categorylinks.cl_sortkey as char) as childs ' \
            'from categorylinks ' \
            'where categorylinks.cl_type = "page"; '

c.execute(sql)
df = []
for row in c.fetchall():
    df.append(row)
    
df = pd.DataFrame(df, columns=["cl_from","cat_title", "childs"])

rows = category_df.loc[category_df.cat_id.isin(no_child), "cat_title"]
rows = pd.DataFrame(rows, columns=["cat_title"]).reset_index(drop=True)

In [11]:
rows.head()

,cat_title
0,Antik_Yunan_kap_şekilleri
1,Vinçler
2,Alman_mucitler
3,Çinli_mucitler
4,"Montgomery,_Alabama_doğumlular"


In [12]:
df.head()

,cl_from,cat_title,childs
0,10,1162_doğumlular,CENGIZ HAN
1,10,1227_yılında_ölenler,CENGIZ HAN
2,10,Cengiz_Han,\nCENGIZ HAN
3,10,ISBN_sihirli_bağlantısını_kullanan_sayfalar,CENGIZ HAN
4,10,Kullanımdan_kaldırılmış_parametreli_kaynak_şab...,CENGIZ HAN\nCENGIZ HAN


In [14]:
result = df[df.cat_title.isin(rows.cat_title)]
#result.to_csv("all_childs.csv", index=False)

In [15]:
print(df.shape, rows.shape, result.shape)

(1983204, 3) (209558, 1) (1211742, 3)


In [16]:
result.head()

,cl_from,cat_title,childs
0,10,1162_doğumlular,CENGIZ HAN
1,10,1227_yılında_ölenler,CENGIZ HAN
2,10,Cengiz_Han,\nCENGIZ HAN
3,10,ISBN_sihirli_bağlantısını_kullanan_sayfalar,CENGIZ HAN
4,10,Kullanımdan_kaldırılmış_parametreli_kaynak_şab...,CENGIZ HAN\nCENGIZ HAN


In [17]:
result[result.cat_title == "Alman_arkeologlar"]

,cl_from,cat_title,childs
21690,14239,Alman_arkeologlar,MANFRED KORFMANN
53001,36112,Alman_arkeologlar,"WIEGAND, THEODOR\nTHEODOR WIEGAND"
53223,36202,Alman_arkeologlar,"SCHLIEMANN, HEINRICH\nHEINRICH SCHLIEMANN"
206086,186047,Alman_arkeologlar,"BLUM, STEPHAN\nSTEPHAN BLUM"
528275,552950,Alman_arkeologlar,ERNST EMIL HERZFELD
808042,1133838,Alman_arkeologlar,WOLFGANG MÜLLER-WIENER
1172732,1577892,Alman_arkeologlar,EBERHARD ZANGGER
1379062,1751796,Alman_arkeologlar,KLAUS SCHMIDT (ARKEOLOG)
1483820,1889251,Alman_arkeologlar,MAX VON OPPENHEIM
1609261,2082912,Alman_arkeologlar,"KRAUTHEIMER, RICHARD\nRICHARD KRAUTHEIMER"


In [18]:
category_df[category_df.cat_title == "Alman_arkeologlar"]

,cat_id,cat_title,parent_cat_id
48607,7899,Alman_arkeologlar,5156869
325710,7899,Alman_arkeologlar,32640
369853,7899,Alman_arkeologlar,19280


In [19]:
merged_df = pd.merge(category_df, result, how = "inner", on = "cat_title")

In [21]:
merged_df.head(10)

,cat_id,cat_title,parent_cat_id,cl_from,childs
0,3830702,Antik_Yunan_kap_şekilleri,8562,475449,HYDRIA
1,3830702,Antik_Yunan_kap_şekilleri,8562,2071158,NESTOR'UN KADEHI
2,3830702,Antik_Yunan_kap_şekilleri,8562,2324224,RITON
3,3830702,Antik_Yunan_kap_şekilleri,8570,475449,HYDRIA
4,3830702,Antik_Yunan_kap_şekilleri,8570,2071158,NESTOR'UN KADEHI
5,3830702,Antik_Yunan_kap_şekilleri,8570,2324224,RITON
6,1742828,Vinçler,8573,192466,\nVINÇ
7,1742828,Vinçler,8573,474534,KULE VINÇ
8,1742828,Vinçler,8573,1437716,AYIRMA TAŞIYICISI
9,1742828,Vinçler,4645829,192466,\nVINÇ


In [23]:
category_df[category_df.cat_id.isin([8570, 8562])]

,cat_id,cat_title,parent_cat_id
7,8562,Antik_Yunan_çömlekçiliği,8573
162146,8562,Antik_Yunan_çömlekçiliği,8571
162166,8570,Antik_Yunanistan'da_mutfak_kültürü,8554
394398,8562,Antik_Yunan_çömlekçiliği,28872
455631,8570,Antik_Yunanistan'da_mutfak_kültürü,19700
461172,8562,Antik_Yunan_çömlekçiliği,21725
461268,8562,Antik_Yunan_çömlekçiliği,684706
464528,8570,Antik_Yunanistan'da_mutfak_kültürü,52307
